# 여름데이터 전처리 진행

## 함수형 코드 진행

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
from scipy import stats

# imputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 계절 이름
my_season = 'summer'

In [3]:
# load data
data = pd.read_csv('../data/'+ my_season +'_train.csv')
test = pd.read_csv('../data/' + my_season + '_test.csv')
test_data = test.copy()

In [4]:
# 변수간의 상관관계가 있는지 확인하고자 하는 함수
def correlation_cal(data, ycol, xcol):
    df = data.copy()
    df = df.dropna(axis = 0)
    select = []
    for names in xcol:
        if df[names].dtypes in ['int64', 'float64']:
            c, p = stats.pearsonr(df[names], df[ycol])
            if p <= 0.05 and abs(c) >= 0.5:
                select.append(names)
    
    return select + [ycol]

In [5]:
# 이상치 탐색하기
def abnormal_data(data, ycol, data_type, Q1, Q2, Q3):
    df = data.copy()
    if data_type == "train":
        Q1 = np.quantile(df[ycol][df[ycol].notnull()], 0.25)
        Q2 = np.quantile(df[ycol][df[ycol].notnull()], 0.5)
        Q3 = np.quantile(df[ycol][df[ycol].notnull()], 0.75)
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df, Q1, Q2, Q3
    
    elif data_type == "test":
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df
    
    else:
        print("없는 데이터 타입입니다. 확인하시고 train과 test중 하나를 입력해주세요.")

In [6]:
# imputation 적용하기 : 두 가지 방법을 활용해 적용해두기
# fit한 모델을 그대로 가져와 test셋에도 똑같이 적용하기 위해 return에 포함시킨다.
def imputation_method(method, data, cols, neightbor = 2, weight = "uniform"):
    if method == "mice":
        imputer_mice = IterativeImputer(random_state=42)
        imputer_mice.fit(data[cols])

        # 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
        data[cols] = pd.DataFrame(imputer_mice.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_mice, data
    
    elif method == "knn":
        # KNN을 이용한 자동 대치
        imputer_KNN = KNNImputer(n_neighbors=neightbor, weights=weight)
        imputer_KNN.fit(data[cols])

        # 데이터 변환
        data[cols] = pd.DataFrame(imputer_KNN.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_KNN, data
    
    else:
        # 다른 임퓨터는 없어요
        print("그 임퓨터는 지원되지 않습니다. 이름을 확인하거나 소문자로 입력 되었는지 확인해주소.")

In [7]:
# 원하는 변수의 평균값 구해두기
def mean_number(data, needs):
    df = data.copy()
    mean_list = df[needs].mean()
    
    for name in tqdm.tqdm(needs):
        df[name][df[name].isna()] = mean_list[name]
    
    print(df[needs].isna().sum())
    return df, mean_list

In [8]:
# 일사, 일조량 데이터 결측값 대치
def sun_power(selectdata, wantcol, hour, sunrise, sunset, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            ((df[hour] < df[sunrise]) | (df[hour] > df[sunset])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (df[weather].isin(['C'])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (~df[weather].isin(['C']))
        ]
        choices = [
            df[name], 
            0, 
            round(df[name][df[weather].isin(['C'])].mean(), 1),
            round(df[name][~df[weather].isin(['C'])].mean(), 1)
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

In [9]:
# 강수량, 강수유무 결측값 대치
def rain_power(selectdata, wantcol, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            (df[weather].isin(['R'])),
            (df[weather].isin(['F', 'H'])),
            (~df[weather].isin(['R', 'F', 'H'])),
        ]
        choices = [
            df[name], 
            round(df[name][df[weather].isin(['R'])].mean(), 1),
            round(df[name][df[weather].isin(['F', 'H'])].mean(), 1),
            round(df[name][~df[weather].isin(['R', 'F', 'H'])].mean(), 1),
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

## 적용 코드

### Method I : 날씨에 따른 결측값 대치 방법

In [10]:
# 기온 이슬점 습도 풍속 대치
data2, mlist = mean_number(data, ['ta', 'td', 'hm', 'ws'])

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 323.52it/s]

ta    0
td    0
hm    0
ws    0
dtype: int64


In [11]:
# 태양과 관련된 변수 대치
data3, param_s = sun_power(data2, ['si', 'ss'], 'hh', 'sunriseh', 'sunseth', 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 52.27it/s]


In [12]:
# 비와 관련된 변수 대치
data4, param_r = rain_power(data3, ['rn', 're'], 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 28.28it/s]


In [13]:
# 봄이므로 적설량 없다 취급
data4['sn'][data4['sn'].isna()] = 0

In [14]:
data4.columns

Index(['stn', 'year', 'ta', 'td', 'hm', 'ws', 'rn', 're', 'ww', 'ts', 'si',
       'ss', 'sn', 'mm', 'dd', 'hh', 'sunriseh', 'sunrisem', 'sunseth',
       'sunsetm'],
      dtype='object')

In [15]:
# 이상치를 결측치로 전환과 동시에 Q1, Q2, Q3 값 뽑아주기
# test 데이테에서 또한 사용해야함
data5, Q1, Q2, Q3 = abnormal_data(data4, "ts", "train", 0, 0, 0)

In [16]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
data5['ss'] = round(data5['ss'] * 60)

In [17]:
# 필요없는 변수 빼기
data6 = data5[data5.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww', 'Unnamed: 0', 'sn'])]

In [18]:
# 상관계수 절댓값 0.5이상과 p-value가 0.05이하인 변수만 선택
# select_col = correlation_cal(data6, 'ts', list(data6.columns.difference(['ts'])))

In [19]:
# imputation 진행하기
# 연도 제외시키기
tmp = data6.drop(['year'],axis = 1)

# mice, data7 = imputation_method("mice", data6, select_col)
# knn, data8 = imputation_method("knn", data6, select_col)
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(tmp)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
tmp2 = pd.DataFrame(imputer_mice.transform(tmp), columns=tmp.columns)

# 데이터에 mice한 값 집어넣기
data6['ts'] = tmp2['ts']

In [20]:
# 데이터 저장
data6.to_csv('../data/' + my_season + '_impute.csv', index = False)

#### 봄, 여름 변수 추가 반영 데이터

- MAE를 비교하였을 때 봄과 여름이 MAE값의 하향세를 보였기 때문에 두 계절에서만 변수를 추가하는 구간

In [21]:
# 계산식 활용해서 변수 추가
data7 = data6.copy()

In [22]:
# 체감온도(℃)
data7['sensory_temp'] = 13.12 + 0.6215*data7['ta'] - 11.37*(data7['ws']**0.16) + 0.3965*(data7['ws']**0.16)*(data7['ta'])

In [23]:
# 온도에 따른 포화 수증기압(mb)
data7['vapor_pressure'] = 6.11*10**((7.5*data7['ta'])/(237.3 + data7['ta']))

In [24]:
# 현재 대기의 수증기압(mb)
data7['air_pressure'] = data7['hm'] * data7['vapor_pressure'] / 100

In [25]:
# 절대습도(g/m^3)
data7['abs_hm'] = (0.794*data7['air_pressure'])/(1+0.00366*data7['ta'])

In [26]:
# 공기중의 혼합비(mb)
data7['air_mix'] = 0.622 * data7['air_pressure']/(1013.25 - data7['air_pressure'])

In [27]:
# 넷째자리까지 반올림해서 저장
round(data7, 4).to_csv('../data/' + my_season + '_new_value.csv', index = False)

#### Method II : 계절마다 월 단위로 평균값을 대치하는 방법

In [28]:
# 맑은 날씨와 아닌 날씨의 구분
data['sun'] = 0
data['sun'][data['ww'] == "C"] = 1

In [29]:
# 비 / 안개, 박무 / 그 외 날씨로 구분
data['rain'] = 0
data['rain'][data['ww'] == "R"] = 2
data['rain'][data['ww'].isin(['F', 'H'])] = 1

In [30]:
# 일몰 이후와 일출 이전 시간의 일사 및 일조는 0
data['si'][((data['hh'] < data['sunriseh']) | (data['hh'] > data['sunseth'])) & (data['si'].isna())] = 0
data['ss'][((data['hh'] < data['sunriseh']) | (data['hh'] > data['sunseth'])) & (data['ss'].isna())] = 0

In [31]:
# 각 집단의 편균값 계산
month = data.groupby(['mm']).mean().reset_index()[['mm', 'ta', 'td', 'hm', 'ws']]
sun = data.groupby(['mm', 'sun']).mean().reset_index()[['mm', 'sun', 'si', 'ss']]
rain = data.groupby(['mm', 'rain']).mean().reset_index()[['mm', 'rain', 'rn', 're']]

In [32]:
# 각 데이터베이스의 이름 중복되지 않게 'mean_' 붙여주기
month.columns = list(map(lambda x: 'mean_' + x if x != 'mm' else x, month.columns))
sun.columns = list(map(lambda x: 'mean_' + x if x not in ['mm', 'sun'] else x, sun.columns))
rain.columns = list(map(lambda x: 'mean_' + x if x not in ['mm', 'rain'] else x, rain.columns))

In [33]:
# 전부 반영해서 합쳐주기
tmp001 = pd.merge(data, month, how = 'left', on = ['mm'])
tmp002 = pd.merge(tmp001, sun, how = 'left', on = ['mm', 'sun'])
mergedata = pd.merge(tmp002, rain, how = 'left', on = ['mm', 'rain'])

In [34]:
# 결측값 대치하는 구간
mergedata['ta'][mergedata['ta'].isna()] = mergedata['mean_ta'][mergedata['ta'].isna()]
mergedata['td'][mergedata['td'].isna()] = mergedata['mean_td'][mergedata['td'].isna()]
mergedata['hm'][mergedata['hm'].isna()] = mergedata['mean_hm'][mergedata['hm'].isna()]
mergedata['ws'][mergedata['ws'].isna()] = mergedata['mean_ws'][mergedata['ws'].isna()]
mergedata['rn'][mergedata['rn'].isna()] = mergedata['mean_rn'][mergedata['rn'].isna()]
mergedata['re'][mergedata['re'].isna()] = mergedata['mean_re'][mergedata['re'].isna()]
mergedata['si'][mergedata['si'].isna()] = mergedata['mean_si'][mergedata['si'].isna()]
mergedata['ss'][mergedata['ss'].isna()] = mergedata['mean_ss'][mergedata['ss'].isna()]

In [35]:
# 이상치를 결측치로 전환과 동시에 Q1, Q2, Q3 값 뽑아주기
# test 데이테에서 또한 사용해야함
mergedata2, Q1, Q2, Q3 = abnormal_data(mergedata, "ts", "train", 0, 0, 0)

In [36]:
# 필요한 변수만 선정하여 뽑아내주기
mergedata3 = mergedata2[['stn', 'ta', 'td', 'hm', 'ws', 'rn', 're', 'ts', 'si', 'ss', 'mm', 'dd', 'hh']]

In [37]:
# mice
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(mergedata3)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
mergedata4 = pd.DataFrame(imputer_mice.transform(mergedata3), columns=mergedata3.columns)

In [38]:
# 잠시 제외했던 year 변수 다시 반영
mergedata4['year'] = data['year']

#### 봄, 여름 변수 추가 반영 데이터

In [39]:
# 계산식 활용해서 변수 추가
mergedata5 = mergedata4.copy()

In [40]:
# 체감온도(℃)
mergedata5['sensory_temp'] = 13.12 + 0.6215*mergedata5['ta'] - 11.37*(mergedata5['ws']**0.16) + 0.3965*(mergedata5['ws']**0.16)*(mergedata5['ta'])

In [41]:
# 온도에 따른 포화 수증기압(mb)
mergedata5['vapor_pressure'] = 6.11*10**((7.5*mergedata5['ta'])/(237.3 + mergedata5['ta']))

In [42]:
# 현재 대기의 수증기압(mb)
mergedata5['air_pressure'] = mergedata5['hm'] * mergedata5['vapor_pressure'] / 100

In [43]:
# 절대습도(g/m^3)
mergedata5['abs_hm'] = (0.794*mergedata5['air_pressure'])/(1+0.00366*mergedata5['ta'])

In [44]:
# 공기중의 혼합비(mb)
mergedata5['air_mix'] = 0.622 * mergedata5['air_pressure']/(1013.25 - mergedata5['air_pressure'])

In [45]:
# 넷째자리까지 반올림해서 저장
round(mergedata5, 4).to_csv('../data/summer_merge_new_value_2.csv', index = False)

### Method III : MICE 방법을 두번 적용하는 구간

In [46]:
# 첫번째 mice
micedata = data[['ta', 'td', 'hm', 'ws', 'sun', 'rain', 'rn', 're', 'si', 'ss', 'mm', 'dd', 'hh']]

In [47]:
# mice
first_imputer_mice = IterativeImputer(random_state=42)
first_imputer_mice.fit(micedata)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
micedata2 = pd.DataFrame(first_imputer_mice.transform(micedata), columns=micedata.columns)

In [48]:
# ts를 다른 값의 대치가 이루어진 후 다시 반영
micedata2['ts'] = data['ts']

In [49]:
# mice
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(micedata2)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
micedata3 = pd.DataFrame(imputer_mice.transform(micedata2), columns=micedata2.columns)

In [50]:
# 빼두었던 지역과 연도 다시 반영해주기
micedata3['year'] = data['year']
micedata3['stn'] = data['stn']

#### 봄, 여름 변수 추가 반영 데이터

In [51]:
# 체감온도(℃)
micedata3['sensory_temp'] = 13.12 + 0.6215*micedata3['ta'] - 11.37*(micedata3['ws']**0.16) + 0.3965*(micedata3['ws']**0.16)*(micedata3['ta'])

In [52]:
# 온도에 따른 포화 수증기압(mb)
micedata3['vapor_pressure'] = 6.11*10**((7.5*micedata3['ta'])/(237.3 + micedata3['ta']))

In [53]:
# 현재 대기의 수증기압(mb)
micedata3['air_pressure'] = micedata3['hm'] * micedata3['vapor_pressure'] / 100

In [54]:
# 절대습도(g/m^3)
micedata3['abs_hm'] = (0.794*micedata3['air_pressure'])/(1+0.00366*micedata3['ta'])

In [55]:
# 공기중의 혼합비(mb)
micedata3['air_mix'] = 0.622 * micedata3['air_pressure']/(1013.25 - micedata3['air_pressure'])

In [56]:
# 넷째자리까지 반올림해서 저장
round(micedata3, 4).to_csv('../data/' + my_season + '_mice_twice_train.csv', index = False)

## Test data

In [57]:
# 값 대치
for name in tqdm.tqdm(['ta', 'td', 'hm', 'ws']):
        test_data[name][test_data[name].isna()] = mlist[name]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 347.95it/s]


In [58]:
# 태양관련 변수 대입
for name in tqdm.tqdm(['si', 'ss']):
    conditions = [
        test_data[name].notnull(),
        ((test_data['hh'] < test_data['sunriseh']) | (test_data['hh'] > test_data['sunseth'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (test_data['ww'].isin(['C'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (~test_data['ww'].isin(['C']))
    ]
    param_s[0] = test_data[name]
    test_data[name] = np.select(conditions, param_s)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 177.55it/s]


In [59]:
# 비 관련 변수 대입
for name in tqdm.tqdm(['rn', 're']):
    conditions = [
        test_data[name].notnull(),
        (test_data['ww'].isin(['R'])),
        (test_data['ww'].isin(['F', 'H'])),
        (~test_data['ww'].isin(['R', 'F', 'H'])),
    ]
    param_r[0] = test_data[name]
    test_data[name] = np.select(conditions, param_r)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 238.67it/s]


In [60]:
# 적설량 0으로 통일
# 여름이므로 적설량 없다 취급
test_data['sn'][test_data['sn'].isna()] = 0

In [61]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
test_data['ss'] = round(test_data['ss'] * 60)

In [62]:
# 필요없는 변수 빼기
test_data2 = test_data[test_data.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww'])]

In [63]:
# 데이터 저장
test_data2.to_csv('../data/' + my_season + '_test_complete.csv', index = False)

===================================================================================================

In [64]:
# test
test_data3 = test_data2.copy()

In [65]:
# 체감온도(℃)
test_data3['sensory_temp'] = 13.12 + 0.6215*test_data3['ta'] - 11.37*(test_data3['ws']**0.16) + 0.3965*(test_data3['ws']**0.16)*(test_data3['ta'])

In [66]:
# 온도에 따른 포화 수증기압(mb)
test_data3['vapor_pressure'] = 6.11*10**((7.5*test_data3['ta'])/(237.3 + test_data3['ta']))

In [67]:
# 현재 대기의 수증기압(mb)
test_data3['air_pressure'] = test_data3['hm'] * test_data3['vapor_pressure'] / 100

In [68]:
# 절대습도(g/m^3)
test_data3['abs_hm'] = (0.794*test_data3['air_pressure'])/(1+0.00366*test_data3['ta'])

In [69]:
# 공기중의 혼합비(mb)
test_data3['air_mix'] = 0.622 * test_data3['air_pressure']/(1013.25 - test_data3['air_pressure'])

In [70]:
# 넷째자리까지 반올림해서 저장
round(test_data3, 4).to_csv('../data/' + my_season + '_new_test_value.csv', index = False)

===================================================================================================

In [71]:
test['sun'] = 0
test['sun'][test['ww'] == "C"] = 1

In [72]:
test['rain'] = 0
test['rain'][test['ww'] == "R"] = 2
test['rain'][test['ww'].isin(['F', 'H'])] = 1

In [73]:
tmp0001 = pd.merge(test, month, how = 'left', on = ['mm'])
tmp0002 = pd.merge(tmp0001, sun, how = 'left', on = ['mm', 'sun'])
mergetest = pd.merge(tmp0002, rain, how = 'left', on = ['mm', 'rain'])

In [74]:
mergetest['si'][((mergetest['hh'] < mergetest['sunriseh']) | (mergetest['hh'] > mergetest['sunseth'])) & (mergetest['si'].isna())] = 0
mergetest['ss'][((mergetest['hh'] < mergetest['sunriseh']) | (mergetest['hh'] > mergetest['sunseth'])) & (mergetest['ss'].isna())] = 0

In [75]:
mergetest['ta'][mergetest['ta'].isna()] = mergetest['mean_ta'][mergetest['ta'].isna()]
mergetest['td'][mergetest['td'].isna()] = mergetest['mean_td'][mergetest['td'].isna()]
mergetest['hm'][mergetest['hm'].isna()] = mergetest['mean_hm'][mergetest['hm'].isna()]
mergetest['ws'][mergetest['ws'].isna()] = mergetest['mean_ws'][mergetest['ws'].isna()]
mergetest['rn'][mergetest['rn'].isna()] = mergetest['mean_rn'][mergetest['rn'].isna()]
mergetest['re'][mergetest['re'].isna()] = mergetest['mean_re'][mergetest['re'].isna()]
mergetest['si'][mergetest['si'].isna()] = mergetest['mean_si'][mergetest['si'].isna()]
mergetest['ss'][mergetest['ss'].isna()] = mergetest['mean_ss'][mergetest['ss'].isna()]

In [76]:
mergetest2 = mergetest[['stn', 'year', 'ta', 'td', 'hm', 'ws', 'rn', 're', 'si', 'ss', 'mm', 'dd', 'hh']]

In [77]:
# 계산식 활용해서 변수 추가
mergetest3 = mergetest2.copy()

In [78]:
# 체감온도(℃)
mergetest3['sensory_temp'] = 13.12 + 0.6215*mergetest3['ta'] - 11.37*(mergetest3['ws']**0.16) + 0.3965*(mergetest3['ws']**0.16)*(mergetest3['ta'])

In [79]:
# 온도에 따른 포화 수증기압(mb)
mergetest3['vapor_pressure'] = 6.11*10**((7.5*mergetest3['ta'])/(237.3 + mergetest3['ta']))

In [80]:
# 현재 대기의 수증기압(mb)
mergetest3['air_pressure'] = mergetest3['hm'] * mergetest3['vapor_pressure'] / 100

In [81]:
# 절대습도(g/m^3)
mergetest3['abs_hm'] = (0.794*mergetest3['air_pressure'])/(1+0.00366*mergetest3['ta'])

In [82]:
# 공기중의 혼합비(mb)
mergetest3['air_mix'] = 0.622 * mergetest3['air_pressure']/(1013.25 - mergetest3['air_pressure'])

In [83]:
# 넷째자리까지 반올림해서 저장
round(mergetest3, 4).to_csv('../data/summer_merge_new_test_value_2.csv', index = False)

===================================================================================================

In [84]:
test['sun'] = 0
test['sun'][test['ww'] == "C"] = 1

In [85]:
test['rain'] = 0
test['rain'][test['ww'] == "R"] = 2
test['rain'][test['ww'].isin(['F', 'H'])] = 1

In [86]:
# 첫번째 mice
testmicedata = test[['ta', 'td', 'hm', 'ws', 'sun', 'rain', 'rn', 're', 'si', 'ss', 'mm', 'dd', 'hh']]

In [87]:
testmicedata2 = pd.DataFrame(first_imputer_mice.transform(testmicedata), columns=testmicedata.columns)

In [88]:
testmicedata2['year'] = test['year']
testmicedata2['stn'] = test['stn']

In [89]:
testmicedata3 = testmicedata2[testmicedata2.columns.difference(['sun', 'rain'])]

In [90]:
# 체감온도(℃)
testmicedata2['sensory_temp'] = 13.12 + 0.6215*testmicedata2['ta'] - 11.37*(testmicedata2['ws']**0.16) + 0.3965*(testmicedata2['ws']**0.16)*(testmicedata2['ta'])

In [91]:
# 온도에 따른 포화 수증기압(mb)
testmicedata2['vapor_pressure'] = 6.11*10**((7.5*testmicedata2['ta'])/(237.3 + testmicedata2['ta']))

In [92]:
# 현재 대기의 수증기압(mb)
testmicedata2['air_pressure'] = testmicedata2['hm'] * testmicedata2['vapor_pressure'] / 100

In [93]:
# 절대습도(g/m^3)
testmicedata2['abs_hm'] = (0.794*testmicedata2['air_pressure'])/(1+0.00366*testmicedata2['ta'])

In [94]:
# 공기중의 혼합비(mb)
testmicedata2['air_mix'] = 0.622 * testmicedata2['air_pressure']/(1013.25 - testmicedata2['air_pressure'])

In [95]:
# 넷째자리까지 반올림해서 저장
round(testmicedata2, 4).to_csv('../data/' + my_season + '_mice_twice_test.csv', index = False)